# RGU-IIT CBR Retrieval Simulator

## Import Libraries

In [2]:
!pip install datasets

from tqdm import tqdm
from typing import *
from datasets import load_dataset, Dataset, DatasetDict
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np
import requests
import pandas as pd
import json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00


## CBR system

In [3]:
class Case:
    def __init__(self, index, question, matching_question_embeddings, text, retrieval_text_embeddings, keywords, retrieval_keywords_embeddings, answer):
        self.index = index
        self.question = question
        self.matching_question_embeddings = np.array(eval(matching_question_embeddings))
        self.text = text
        self.retrieval_text_embeddings = np.array(eval(retrieval_text_embeddings))
        self.keywords = keywords
        self.retrieval_keywords_embeddings = np.array(eval(retrieval_keywords_embeddings))
        self.answer = answer

case_database = []

class CbrSystem:
    def __init__(self, cases):
        self.cases = cases
        self.global_matching_question_embeddings = []
        self.global_retrieval_text_embeddings = []
        self.global_retrieval_keyword_embeddings = []
        cases = cases
        for case in self.cases:
            self.global_matching_question_embeddings.append(case.matching_question_embeddings)
            self.global_retrieval_text_embeddings.append(case.retrieval_text_embeddings)
            self.global_retrieval_keyword_embeddings.append(case.retrieval_keywords_embeddings)

    def retrieve_matches(self, matching_query_embeddings, retrieval_query_embeddings, document_count, question_weight, snippet_weight, keywords_weight):
        question_similarities = np.zeros(len(self.cases))
        snippet_similarities = np.zeros(len(self.cases))
        keyword_similarities = np.zeros(len(self.cases))

        if question_weight > 0:
            question_similarities = np.array([cosine_similarity([matching_query_embeddings], [matching_question_embedding])[0][0] for matching_question_embedding in self.global_matching_question_embeddings])
        if snippet_weight > 0:
            snippet_similarities = np.array([cosine_similarity([retrieval_query_embeddings], [retrieval_text_embeddings])[0][0] for retrieval_text_embeddings in self.global_retrieval_text_embeddings])
        if keywords_weight > 0:
            keyword_similarities = np.array([cosine_similarity([retrieval_query_embeddings], [retrieval_keyword_embeddings])[0][0] for retrieval_keyword_embeddings in self.global_retrieval_keyword_embeddings])

        combined_similarities = (question_weight * np.array(question_similarities)) + (snippet_weight * np.array(snippet_similarities)) + (keywords_weight * np.array(keyword_similarities))
        closest_case_indices = np.argsort(combined_similarities)[-document_count:][::-1]

        results = [self.cases[index] for index in closest_case_indices]
        return results, closest_case_indices

## Load assets from pre-built sources

In [5]:
df = pd.read_csv("resources/main.csv")
selected_columns = [
                    'question', 'answer', 'snippet', 'keywords',
                    'question_normal_bert_matching_embeddings',
                    'question_legal_bert_matching_embeddings',
                    'question_angle_bert_matching_embeddings',
                    'question_normal_bert_retrieval_embeddings',
                    'question_legal_bert_retrieval_embeddings',
                    'question_angle_bert_retrieval_embeddings',
                    'answer_normal_bert_matching_embeddings',
                    'answer_legal_bert_matching_embeddings',
                    'answer_angle_bert_matching_embeddings',
                    'answer_normal_bert_retrieval_embeddings',
                    'answer_legal_bert_retrieval_embeddings',
                    'answer_angle_bert_retrieval_embeddings',
                    'keyword_normal_bert_matching_embeddings',
                    'keyword_legal_bert_matching_embeddings',
                    'keyword_angle_bert_matching_embeddings',
                    'keyword_normal_bert_retrieval_embeddings',
                    'keyword_legal_bert_retrieval_embeddings',
                    'keyword_angle_bert_retrieval_embeddings',
                    'snippet_normal_bert_matching_embeddings',
                    'snippet_legal_bert_matching_embeddings',
                    'snippet_angle_bert_matching_embeddings',
                    'snippet_normal_bert_retrieval_embeddings',
                    'snippet_legal_bert_retrieval_embeddings',
                    'snippet_angle_bert_retrieval_embeddings'
                    ]
main_df = df.loc[:, selected_columns].drop_duplicates(subset='question').reset_index(drop=True)
print(len(main_df))
df_test = pd.read_csv("main_test.csv")
print(len(df_test))

2084
35


In [6]:
cases_normal_bert = []
cases_legal_bert = []
cases_angle_bert = []

for index, row in tqdm(main_df.iterrows()):
  cases_normal_bert.append(Case(index, row['question'], row['question_normal_bert_matching_embeddings'], row['snippet'], row['snippet_normal_bert_retrieval_embeddings'], row['keywords'], row['keyword_normal_bert_retrieval_embeddings'], row['answer']))
  cases_legal_bert.append(Case(index, row['question'], row['question_legal_bert_matching_embeddings'], row['snippet'], row['snippet_legal_bert_retrieval_embeddings'], row['keywords'], row['keyword_legal_bert_retrieval_embeddings'], row['answer']))
  cases_angle_bert.append(Case(index, row['question'], row['question_angle_bert_matching_embeddings'], row['snippet'], row['snippet_angle_bert_retrieval_embeddings'], row['keywords'], row['keyword_angle_bert_retrieval_embeddings'], row['answer']))

normal_bert_cbr_system = CbrSystem(cases_normal_bert)
legal_bert_cbr_system = CbrSystem(cases_legal_bert)
angle_bert_cbr_system = CbrSystem(cases_angle_bert)

print('\n Cases loaded into the systems');

2084it [01:30, 22.90it/s]


 Cases loaded into the systems


## Algorithms

In [7]:
# Pipeline 2 - RAG on question
def execute_pipeline_2(cbr_system, question, question_matching_embeddings, question_retrieval_embeddings, k):
    matched_cases, indexes = cbr_system.retrieve_matches(np.array(eval(question_matching_embeddings)), np.array(eval(question_retrieval_embeddings)), k, 1, 0, 0)
    return indexes

# Pipeline 4 - RAG on snippet
def execute_pipeline_4(cbr_system, question, question_matching_embeddings, question_retrieval_embeddings, k):
    matched_cases, indexes = cbr_system.retrieve_matches(np.array(eval(question_matching_embeddings)), np.array(eval(question_retrieval_embeddings)), k, 0, 1, 0)
    return indexes

# Pipeline 6 - RAG on keywords / entities
def execute_pipeline_6(cbr_system, question, question_matching_embeddings, question_retrieval_embeddings, k):
    matched_cases, indexes = cbr_system.retrieve_matches(np.array(eval(question_matching_embeddings)), np.array(eval(question_retrieval_embeddings)), k, 0, 0, 1)
    return indexes

# Pipeline X - RAG on question / keywords / entities with dynamic weights [Used for 8, 10, 12 pipelines]
def execute_pipeline_x(cbr_system, question, question_matching_embeddings, question_retrieval_embeddings, k, w1, w2, w3):
    matched_cases, indexes = cbr_system.retrieve_matches(np.array(eval(question_matching_embeddings)), np.array(eval(question_retrieval_embeddings)), k, w1, w2, w3)
    return indexes

# Run simulator

In [8]:
k = 2084

In [ ]:
new_df = pd.DataFrame(columns=['index'])

i = 0
for index, row in df_test.iterrows():
    row['case_index'] = index

    row['normal_bert_pipeline_2_case_indexes'] = execute_pipeline_2(normal_bert_cbr_system, row['question'], row['question_normal_bert_matching_embeddings'], row['question_normal_bert_retrieval_embeddings'], k)
    row['normal_bert_pipeline_4_case_indexes'] = execute_pipeline_4(normal_bert_cbr_system, row['question'], row['question_normal_bert_matching_embeddings'], row['question_normal_bert_retrieval_embeddings'], k)
    row['normal_bert_pipeline_6_case_indexes'] = execute_pipeline_6(normal_bert_cbr_system, row['question'], row['question_normal_bert_matching_embeddings'], row['question_normal_bert_retrieval_embeddings'], k)
    row['normal_bert_pipeline_8_case_indexes'] = execute_pipeline_x(normal_bert_cbr_system, row['question'], row['question_normal_bert_matching_embeddings'], row['question_normal_bert_retrieval_embeddings'], k, 0.2, 0.4, 0.4)
    row['normal_bert_pipeline_10_case_indexes'] = execute_pipeline_x(normal_bert_cbr_system, row['question'], row['question_normal_bert_matching_embeddings'], row['question_normal_bert_retrieval_embeddings'], k, 0.3, 0.4, 0.3)
    row['normal_bert_pipeline_12_case_indexes'] = execute_pipeline_x(normal_bert_cbr_system, row['question'], row['question_normal_bert_matching_embeddings'], row['question_normal_bert_retrieval_embeddings'], k, 0.25, 0.4, 0.35)

    row['legal_bert_pipeline_2_case_indexes'] = execute_pipeline_2(legal_bert_cbr_system, row['question'], row['question_legal_bert_matching_embeddings'], row['question_legal_bert_retrieval_embeddings'], k)
    row['legal_bert_pipeline_4_case_indexes'] = execute_pipeline_4(legal_bert_cbr_system, row['question'], row['question_legal_bert_matching_embeddings'], row['question_legal_bert_retrieval_embeddings'], k)
    row['legal_bert_pipeline_6_case_indexes'] = execute_pipeline_6(legal_bert_cbr_system, row['question'], row['question_legal_bert_matching_embeddings'], row['question_legal_bert_retrieval_embeddings'], k)
    row['legal_bert_pipeline_8_case_indexes'] = execute_pipeline_x(legal_bert_cbr_system, row['question'], row['question_legal_bert_matching_embeddings'], row['question_legal_bert_retrieval_embeddings'], k, 0.2, 0.4, 0.4)
    row['legal_bert_pipeline_10_case_indexes'] = execute_pipeline_x(legal_bert_cbr_system, row['question'], row['question_legal_bert_matching_embeddings'], row['question_legal_bert_retrieval_embeddings'], k, 0.3, 0.4, 0.3)
    row['legal_bert_pipeline_12_case_indexes'] = execute_pipeline_x(legal_bert_cbr_system, row['question'], row['question_legal_bert_matching_embeddings'], row['question_legal_bert_retrieval_embeddings'], k, 0.25, 0.4, 0.35)

    row['angle_bert_pipeline_2_case_indexes'] = execute_pipeline_2(angle_bert_cbr_system, row['question'], row['question_angle_bert_matching_embeddings'], row['question_angle_bert_retrieval_embeddings'], k)
    row['angle_bert_pipeline_4_case_indexes'] = execute_pipeline_4(angle_bert_cbr_system, row['question'], row['question_angle_bert_matching_embeddings'], row['question_angle_bert_retrieval_embeddings'], k)
    row['angle_bert_pipeline_6_case_indexes'] = execute_pipeline_6(angle_bert_cbr_system, row['question'], row['question_angle_bert_matching_embeddings'], row['question_angle_bert_retrieval_embeddings'], k)
    row['angle_bert_pipeline_8_case_indexes'] = execute_pipeline_x(angle_bert_cbr_system, row['question'], row['question_angle_bert_matching_embeddings'], row['question_angle_bert_retrieval_embeddings'], k, 0.2, 0.4, 0.4)
    row['angle_bert_pipeline_10_case_indexes'] = execute_pipeline_x(angle_bert_cbr_system, row['question'], row['question_angle_bert_matching_embeddings'], row['question_angle_bert_retrieval_embeddings'], k, 0.3, 0.4, 0.3)
    row['angle_bert_pipeline_12_case_indexes'] = execute_pipeline_x(angle_bert_cbr_system, row['question'], row['question_angle_bert_matching_embeddings'], row['question_angle_bert_retrieval_embeddings'], k, 0.25, 0.4, 0.35)

    new_df = new_df.append(row)

In [10]:
new_df

,index,question,answer,original_texts,question_normal_bert_matching_embeddings,question_legal_bert_matching_embeddings,question_angle_bert_matching_embeddings,question_normal_bert_retrieval_embeddings,question_legal_bert_retrieval_embeddings,question_angle_bert_retrieval_embeddings,...,legal_bert_pipeline_6_case_indexes,legal_bert_pipeline_8_case_indexes,legal_bert_pipeline_10_case_indexes,legal_bert_pipeline_12_case_indexes,angle_bert_pipeline_2_case_indexes,angle_bert_pipeline_4_case_indexes,angle_bert_pipeline_6_case_indexes,angle_bert_pipeline_8_case_indexes,angle_bert_pipeline_10_case_indexes,angle_bert_pipeline_12_case_indexes
0,NaN,In the case Econ Holdings Pty Ltd v Sims Lockw...,In the case Econ Holdings Pty Ltd v Sims Lockw...,[114 169],"[0.10423540323972702, -0.1665179282426834, -0....","[0.0027042031288146973, -0.12522736191749573, ...","[-0.3710246682167053, -0.30695846676826477, -0...","[0.09509215503931046, -0.2021857500076294, -0....","[-0.0023545799776911736, -0.10421659797430038,...","[-0.4049617350101471, -0.05314958840608597, -0...",...,"[640, 1451, 741, 1611, 1311, 599, 1897, 1803, ...","[106, 761, 242, 1603, 1311, 1389, 599, 1803, 6...","[106, 1603, 761, 1311, 242, 1800, 1950, 810, 2...","[106, 761, 1603, 242, 1311, 1389, 1950, 258, 5...","[1591, 1259, 264, 1085, 1717, 1926, 1850, 2001...","[1389, 861, 1484, 738, 867, 326, 2029, 2047, 5...","[1897, 640, 1145, 1950, 597, 425, 114, 1213, 4...","[1145, 1950, 640, 597, 1845, 1897, 2047, 1389,...","[1145, 1950, 640, 597, 1845, 1075, 264, 1259, ...","[1145, 1950, 640, 597, 1845, 1075, 2047, 114, ..."
1,NaN,Considering the balance of public interest and...,The details in Events 38832023 and 3860136 inc...,[270 347],"[-0.019197409972548485, -0.0016128727002069354...","[-0.12920181453227997, 0.08838242292404175, 0....","[0.624129056930542, -0.3914183974266052, -0.04...","[0.02651604823768139, -0.013784815557301044, -...","[-0.05422603711485863, 0.1209280863404274, -0....","[0.36398348212242126, -0.16158992052078247, -0...",...,"[20, 1920, 540, 495, 1606, 1528, 1451, 726, 88...","[1920, 1606, 1430, 660, 1451, 495, 934, 1191, ...","[1430, 1606, 1920, 660, 180, 1191, 1451, 934, ...","[1430, 1606, 1920, 660, 1451, 180, 1191, 934, ...","[270, 933, 1291, 1534, 1633, 1243, 366, 501, 3...","[347, 976, 369, 307, 1743, 628, 380, 677, 933,...","[1822, 347, 837, 883, 2007, 687, 371, 206, 123...","[347, 369, 933, 976, 837, 1720, 206, 307, 1648...","[347, 369, 933, 976, 1720, 837, 270, 307, 206,...","[347, 369, 933, 976, 837, 1720, 307, 206, 270,..."
2,NaN,In two separate cases heard in the Supreme Cou...,In the case of Mekhail v Hana; Mekail v Hana; ...,[ 388 1662],"[0.02596280165016651, -0.07136604189872742, -0...","[-0.032598309218883514, 0.283461332321167, 0.0...","[-0.322487473487854, -0.10779520124197006, -0....","[0.06910744309425354, -0.13112026453018188, -0...","[-0.0018813080387189984, 0.2632799446582794, 0...","[-0.3880188763141632, 0.0675685852766037, -0.7...",...,"[124, 1444, 1451, 1417, 741, 551, 744, 1357, 1...","[488, 124, 81, 1367, 805, 1950, 22, 1444, 1897...","[488, 1367, 81, 124, 805, 22, 1950, 1741, 1897...","[488, 81, 124, 1367, 805, 22, 1950, 1444, 1741...","[1850, 591, 334, 1833, 580, 72, 688, 1978, 113...","[867, 1145, 1781, 1144, 1235, 2047, 1263, 1259...","[1145, 1605, 1844, 1213, 2047, 1614, 1519, 113...","[1145, 2047, 1138, 1259, 275, 1844, 1484, 1748...","[1145, 2047, 1138, 1259, 275, 1845, 1263, 1166...","[1145, 2047, 1138, 1259, 275, 1845, 1166, 254,..."
3,NaN,"In the context of the Federal Court Rules, how...","According to the judgment of Gordon J, an amen...",[1881 1881],"[0.05399247631430626, 0.03763565793633461, -0....","[-0.0940106064081192, 0.031954631209373474, 0....","[-0.08438273519277573, -0.07412683963775635, -...","[0.0569213330745697, -0.042861513793468475, -0...","[-0.09847217798233032, 0.03212566673755646, -0...","[-0.20830093324184418, 0.040662266314029694, -...",...,"[1451, 1831, 714, 1417, 1315, 1126, 1858, 794,...","[1881, 810

In [11]:
!huggingface-cli login --token XXX
new_df.reset_index(drop=True, inplace=True)
output_dataset = DatasetDict({'XXX': Dataset.from_pandas(new_df)})
output_dataset.push_to_hub('XXX')